In [1]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

In [2]:
# Manually adding required courses since they were missing in the CSV
required_courses = [
    ('required', [('4199', 0), ('4004', 3), ('4150', 0), ('4106', 3), ('4404', 3), ('ENGIE4000', 0)])
]

analytics = [
    ('1+', [('4523', 0), ('4650', 3)]),
    ('9c', [('4523', 3), ('4650', 3), ('4108', 3), ('4418', 3), ('4526', 3), 
            ('4540', 3), ('4532', 1.5), ('4533', 1.5), ('4530', 3), ('4545', 3)])
]

entrepreneurship = [
    ('1+', [('4200', 3), ('4550', 3), ('4570', 3), ('4998', 3)])
]

FE_M = [
    ('required', [('4700', 3), ('4403', 3)]),
    ('1+', [('4602', 3), ('4620', 3), ('4630', 3), ('4731', 3), ('4732', 3), 
            ('B8112', 3), ('4711', 3), ('4734', 1.5), ('4735', 3), ('B8307', 3)])
]

HM = [
    ('required', [('4507', 3)])
]

L_SCM = [
    ('2+', [('4108', 3), ('4405', 3), ('4418', 3), ('4507', 3), ('4601', 3)]),
    ('at most 1', [('4412', 3), ('4510', 3), ('4520', 3), ('4521', 3), 
                   ('B8107', 3), ('B8109', 3), ('B8123', 3)])
]

ML_AI = [
    ('1+', [('4525', 3), ('4742', 3), ('4540', 3), ('4575', 3)]),
    ('9c', [('4742', 3), ('4575', 3), ('4545', 3), ('4530', 3), ('4579', 1.5), ('4721', 3), ('4525', 3), ('4540', 3), ('4650', 3), ('4523', 3), ('4212', 3), ('4536', 3)]),
    ('at most 1', [('4525', 3), ('4540', 3), ('4650', 3)]),
    ('at most 1', [('4523', 3), ('4212', 3), ('4536', 3)])
]

optim = [
    ('1+', [('6613', 3), ('6614', 3), ('6616', 3)]),
    ('9c', [('4008', 3), ('4108', 3), ('4405', 3), ('4418', 3), ('4505', 3), 
            ('4507', 3), ('4630', 3), ('CSOR 4231', 3), ('4601', 3), 
            ('6613', 3), ('6614', 3), ('EEOR 6616', 3), ('EEAE 4220', 3), ('ECE 4650', 3)])
]

SM = [
    ('1+', [('6711', 3), ('6712', 3), ('8100', 3)]),
    ('1+', [('4601', 3), ('4602', 3), ('4700', 3), ('4745', 3)]),
    ('1+', [('4525', 3), ('4742', 3), ('6617', 3), ('ORCS 4529', 3)])
]

# Combining all courses into a single structure as a list
courses = [
    ('required_courses', required_courses),
    ('analytics', analytics),
    ('entrepreneurship', entrepreneurship),
    ('FE_M', FE_M),
    ('HM', HM),
    ('L_SCM', L_SCM),
    ('ML_AI', ML_AI),
    ('optim', optim),
    ('SM', SM)
]

# Print the structure
for concentration, requirements in courses:
    print(f"{concentration}:")
    for requirement_type, course_list in requirements:
        print(f"  {requirement_type}: {course_list}")


required_courses:
  required: [('4199', 0), ('4004', 3), ('4150', 0), ('4106', 3), ('4404', 3), ('ENGIE4000', 0)]
analytics:
  1+: [('4523', 0), ('4650', 3)]
  9c: [('4523', 3), ('4650', 3), ('4108', 3), ('4418', 3), ('4526', 3), ('4540', 3), ('4532', 1.5), ('4533', 1.5), ('4530', 3), ('4545', 3)]
entrepreneurship:
  1+: [('4200', 3), ('4550', 3), ('4570', 3), ('4998', 3)]
FE_M:
  required: [('4700', 3), ('4403', 3)]
  1+: [('4602', 3), ('4620', 3), ('4630', 3), ('4731', 3), ('4732', 3), ('B8112', 3), ('4711', 3), ('4734', 1.5), ('4735', 3), ('B8307', 3)]
HM:
  required: [('4507', 3)]
L_SCM:
  2+: [('4108', 3), ('4405', 3), ('4418', 3), ('4507', 3), ('4601', 3)]
  at most 1: [('4412', 3), ('4510', 3), ('4520', 3), ('4521', 3), ('B8107', 3), ('B8109', 3), ('B8123', 3)]
ML_AI:
  1+: [('4525', 3), ('4742', 3), ('4540', 3), ('4575', 3)]
  9c: [('4742', 3), ('4575', 3), ('4545', 3), ('4530', 3), ('4579', 1.5), ('4721', 3), ('4525', 3), ('4540', 3), ('4650', 3), ('4523', 3), ('4212', 3), ('4

In [ ]:
# Initialize Gurobi model
m = Model("Concentration_Maximization")

# Set parameters to search for diverse solutions
m.setParam(GRB.Param.PoolSearchMode, 2)  # Enable solution pool
m.setParam(GRB.Param.PoolSolutions, 10000000)  # Set large limit for the number of solutions

# Step 1: Create binary decision variables for courses (x_courseID)
x = {}  # Course decision variables

# Iterate through each concentration and create course decision variables
for concentration, course_requirements in courses:
    for requirement_type, course_list in course_requirements:
        for course_code, credits in course_list:
            var_name = f"x_{course_code}"
            if var_name not in x:  # Ensure each course variable is created only once
                x[var_name] = m.addVar(vtype=GRB.BINARY, name=var_name)

# Step 2: Create binary decision variables for concentrations (z_concentration)
z = {}  # Concentration decision variables

for concentration, _ in courses:
    z[concentration] = m.addVar(vtype=GRB.BINARY, name=f"z_{concentration}")

# Step 3: Mandatory completion of Core and FE_M requirements
core_concentration = "required_courses"
fem_concentration = "FE_M"

# Ensure the `required_courses` and `FE_M` concentrations are always selected
m.addConstr(z[core_concentration] == 1, name="required_courses_selected")
m.addConstr(z[fem_concentration] == 1, name="FE_M_selected")

# Step 4: Link course selection to concentration achievement for other concentrations
for concentration, course_requirements in courses:
    if concentration not in {core_concentration, fem_concentration}:  # Skip core and FE_M
        for requirement_type, course_list in course_requirements:
            if requirement_type == '1+':  # At least 1 course must be selected
                m.addConstr(quicksum(x[f"x_{course_code}"] for course_code, credits in course_list) >= z[concentration], name=f"constr_{concentration}_1plus")
            elif requirement_type == '9c':  # At least 9 credits must be selected
                m.addConstr(quicksum(credits * x[f"x_{course_code}"] for course_code, credits in course_list) >= 9 * z[concentration], name=f"constr_{concentration}_9credits")
            elif requirement_type == 'required':  # All required courses must be selected
                for course_code, credits in course_list:
                    m.addConstr(x[f"x_{course_code}"] == z[concentration], name=f"constr_{concentration}_required_{course_code}")
            elif requirement_type == 'at most 1':  # At most 1 course must be selected
                m.addConstr(quicksum(x[f"x_{course_code}"] for course_code, credits in course_list) <= z[concentration], name=f"constr_{concentration}_at_most_1")

# Step 5: Add a constraint to ensure the total selected credits are exactly 30
credit_limit = 30
m.addConstr(quicksum(credits * x[f"x_{course_code}"] for concentration, course_requirements in courses for requirement_type, course_list in course_requirements for course_code, credits in course_list) == credit_limit, name="credit_limit")

# Step 6: Set the objective to maximize the number of additional concentrations selected
m.setObjective(quicksum(z[concentration] for concentration, _ in courses if concentration not in {core_concentration, fem_concentration}), GRB.MAXIMIZE)

# Step 7: Optimize the model
m.optimize()

# Step 8: Extract and display multiple solutions
solution_count = m.SolCount
print(f"Number of solutions found: {solution_count}")

# Store results in a dictionary where the key is the set of concentrations and value is the list of course sets
results = {}
max_concentrations = 0  # To store the maximum number of concentrations found

for sol in range(solution_count):
    m.setParam(GRB.Param.SolutionNumber, sol)
    
    # Extract selected concentrations and courses for the current solution
    selected_concentrations = tuple(concentration for concentration, _ in courses if z[concentration].Xn > 0.5)
    selected_courses = tuple(course for course in x if x[course].Xn > 0.5)
    
    # Track the number of concentrations in the current solution
    num_concentrations = len(selected_concentrations)
    
    # Update the max number of concentrations found
    if num_concentrations > max_concentrations:
        max_concentrations = num_concentrations
        results = {}  # Reset results to only keep solutions with the highest number of concentrations

    # If the current solution has the max number of concentrations, store it
    if num_concentrations == max_concentrations:
        if selected_concentrations not in results:
            results[selected_concentrations] = []
        results[selected_concentrations].append(selected_courses)

# Step 9: Print only the solutions with the maximum number of concentrations
print(f"\nShowing solutions with {max_concentrations} concentrations:")
for concentrations, courses_sets in results.items():
    print("\nSelected Concentrations:")
    print(", ".join(concentrations))
    
    for course_set in courses_sets:
        print("\n- " + ", ".join(course_set))


Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 10000000
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 73 columns and 151 nonzeros
Model fingerprint: 0xd2ab3453
Variable types: 0 continuous, 73 integer (73 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 1.0000000
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 14 rows, 70 columns, 146 nonzeros
Variable types: 0 continuous, 70 integer (70 binary)
Found heuristic solution: objective 2.0000000
Found heuristic solution: objective 4.0000000

Root relaxation: objective 5.000000e+00, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objec

 623312 111030          -   68               -    4.00000      -   0.2  485s
 630895 112862          -   70               -    4.00000      -   0.2  497s
 633871 112862          -   65               -    4.00000      -   0.2  502s
 638903 114319          -   65               -    4.00000      -   0.2  510s
 641833 114319     cutoff   70               -    4.00000      -   0.2  515s
 646911 115767     cutoff   73               -    4.00000      -   0.2  524s
 646997 115767          -   64               -    4.00000      -   0.2  525s
 653913 115767          -   60               -    4.00000      -   0.2  536s
 659921 116943     cutoff   72               -    4.00000      -   0.2  546s
 662927 118183          -   64               -    4.00000      -   0.2  551s
 670935 119408          -   72               -    4.00000      -   0.2  565s
 673789 119408          -   70               -    4.00000      -   0.2  570s
 678943 120278     cutoff   73               -    4.00000      -   0.2  579s

 986063 165662     cutoff   57               -    4.00000      -   0.1 1335s
 988180 165662     cutoff   53               -    4.00000      -   0.1 1340s
 991255 166329          -   52               -    4.00000      -   0.1 1349s
 991269 166329          -   66               -    4.00000      -   0.1 1350s
 999263 166843     cutoff   59               -    4.00000      -   0.1 1373s
 999647 166843          -   52               -    4.00000      -   0.1 1375s
 1002123 166843          -   71               -    4.00000      -   0.1 1382s
 1004147 166843     cutoff   69               -    4.00000      -   0.1 1388s
 1007271 167917          -   57               -    4.00000      -   0.1 1397s
 1008039 167917     cutoff   56               -    4.00000      -   0.1 1400s
 1011244 167917          -   38               -    4.00000      -   0.1 1408s
 1012189 167917          -   50               -    4.00000      -   0.1 1411s
 1015279 169149     cutoff   59               -    4.00000      -   0.

 1271488 199668     cutoff   67               -    4.00000      -   0.1 2320s
 1279543 201521          -   72               -    4.00000      -   0.1 2350s
 1287551 202874    3.00000   52    1          -    4.00000      -   0.1 2380s
 1289438 202874          -   66               -    4.00000      -   0.1 2387s
 1295559 203922     cutoff   73               -    4.00000      -   0.1 2409s
 1295750 203922     cutoff   75               -    4.00000      -   0.1 2410s
 1297402 203922     cutoff   68               -    4.00000      -   0.1 2416s
 1303567 204746     cutoff   70               -    4.00000      -   0.1 2440s
 1307553 204746          -   64               -    4.00000      -   0.1 2455s
 1311572 204746          -   69               -    4.00000      -   0.1 2469s
 1311662 205493     cutoff   72               -    4.00000      -   0.1 2470s
 1313509 205493     cutoff   67               -    4.00000      -   0.1 2476s
 1314533 205493          -   69               -    4.00000      

 1536551 227748     cutoff   57               -    4.00000      -   0.1 3445s
 1538655 227748          -   56               -    4.00000      -   0.1 3456s
 1543807 228466     cutoff   73               -    4.00000      -   0.1 3484s
 1543824 228466          -   63               -    4.00000      -   0.1 3485s
 1544774 228466     cutoff   69               -    4.00000      -   0.1 3490s
 1549648 228466     cutoff   70               -    4.00000      -   0.1 3515s
 1551815 229462     cutoff   63               -    4.00000      -   0.1 3527s
 1552188 229462          -   52               -    4.00000      -   0.1 3530s
 1553730 229462          -   54               -    4.00000      -   0.1 3538s
 1555716 229462     cutoff   67               -    4.00000      -   0.1 3549s
 1559823 230023     cutoff   72               -    4.00000      -   0.1 3568s
 1559990 230023     cutoff   69               -    4.00000      -   0.1 3570s
 1564787 230023     cutoff   72               -    4.00000      

...

 14332775 1364429          -   72               -    4.00000      -   0.1 995675s
 14332785 1364429          -   65               -    4.00000      -   0.1 995680s
 14332786 1364429          -   66               -    4.00000      -   0.1 995687s
 14332795 1364429          -   68               -    4.00000      -   0.1 995690s
 14332822 1364429     cutoff   69               -    4.00000      -   0.1 995695s
 14332826 1364429          -   67               -    4.00000      -   0.1 995700s
 14332839 1364429     cutoff   70               -    4.00000      -   0.1 995705s
 14332842 1364429     cutoff   70               -    4.00000      -   0.1 995716s
 14332847 1364429          -   69               -    4.00000      -   0.1 995723s
 14332849 1364429     cutoff   70               -    4.00000      -   0.1 995725s
 14332881 1364429     cutoff   70               -    4.00000      -   0.1 995730s
 14332932 1364429          -   67               -    4.00000      -   0.1 995735s
 14332982 136442

 14341265 1364653     cutoff   60               -    4.00000      -   0.1 997425s
 14341281 1364653     cutoff   59               -    4.00000      -   0.1 997430s
 14341313 1364653          -   57               -    4.00000      -   0.1 997435s
 14341349 1364653     cutoff   63               -    4.00000      -   0.1 997440s
 14341405 1364653     cutoff   70               -    4.00000      -   0.1 997445s
 14341449 1364653     cutoff   69               -    4.00000      -   0.1 997450s
 14341464 1364653     cutoff   69               -    4.00000      -   0.1 997455s
 14341500 1364653     cutoff   67               -    4.00000      -   0.1 997460s
 14347532 1364653     cutoff   70               -    4.00000      -   0.1 998463s
 14348538 1364749    4.00000   50    1          -    4.00000      -   0.1 998571s
 14348561 1364749          -   63               -    4.00000      -   0.1 998575s
 14348590 1364749     cutoff   66               -    4.00000      -   0.1 998580s
 14348628 136474

 14372911 1365220          -   65               -    4.00000      -   0.1 1002725s
 14372964 1365220     cutoff   65               -    4.00000      -   0.1 1002730s
 14372995 1365220          -   65               -    4.00000      -   0.1 1002735s
 14373046 1365220     cutoff   70               -    4.00000      -   0.1 1002740s
 14373088 1365220          -   65               -    4.00000      -   0.1 1002745s
 14373142 1365220          -   65               -    4.00000      -   0.1 1002752s
 14373144 1365220          -   66               -    4.00000      -   0.1 1002755s
 14373161 1365220          -   61               -    4.00000      -   0.1 1002760s
 14373175 1365220     cutoff   64               -    4.00000      -   0.1 1002766s
 14373183 1365220     cutoff   66               -    4.00000      -   0.1 1002775s
 14373184 1365220          -   65               -    4.00000      -   0.1 1002781s
 14373187 1365220          -   64               -    4.00000      -   0.1 1002788s
 143

 14396776 1365421     cutoff   70               -    4.00000      -   0.1 1006020s
 14396822 1365421          -   63               -    4.00000      -   0.1 1006025s
 14396859 1365421          -   69               -    4.00000      -   0.1 1006030s
 14396874 1365421          -   69               -    4.00000      -   0.1 1006035s
 14396910 1365421     cutoff   67               -    4.00000      -   0.1 1006040s
 14396947 1365421          -   69               -    4.00000      -   0.1 1006045s
 14396991 1365421     cutoff   67               -    4.00000      -   0.1 1006050s
 14397042 1365421          -   58               -    4.00000      -   0.1 1006055s
 14397084 1365421     cutoff   67               -    4.00000      -   0.1 1006060s
 14397127 1365421          -   62               -    4.00000      -   0.1 1006065s
 14397156 1365421     cutoff   60               -    4.00000      -   0.1 1006070s
 14397193 1365421     cutoff   64               -    4.00000      -   0.1 1006075s
 143

 14413148 1365922     cutoff   63               -    4.00000      -   0.1 1087173s
 14413149 1365922          -   60               -    4.00000      -   0.1 1087175s
 14413153 1365922     cutoff   63               -    4.00000      -   0.1 1087181s
 14413156 1365922     cutoff   63               -    4.00000      -   0.1 1087185s
 14413182 1365922          -   61               -    4.00000      -   0.1 1087190s
 14413209 1365922     cutoff   65               -    4.00000      -   0.1 1087198s
 14413210 1365922          -   64               -    4.00000      -   0.1 1087202s
 14413215 1365922     cutoff   65               -    4.00000      -   0.1 1087205s
 14413232 1365922     cutoff   64               -    4.00000      -   0.1 1087210s
 14413233 1365922          -   63               -    4.00000      -   0.1 1087244s
 14413238 1365922          -   63               -    4.00000      -   0.1 1087245s
 14413258 1365922     cutoff   64               -    4.00000      -   0.1 1087255s
 144